In [1]:
%config Completer.use_jedi = False

import sys
sys.path.insert(1,'../')
sys.path.insert(1,'../models/')


# Pytorch related
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms
from torch.utils import data as dt
from torchinfo import summary
import torchvision.models as pretrained_models
from alexnet_pytorch import AlexNet
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils import model_zoo



import numpy as np
import pandas as pd
import os


# Extra imports
# from pytorch_pretrained_vit import ViT
from lib.feature_extractor import FeatureExtractor
from lib.build_fe_ft_models import *
from models import barlow_twins


updated


# Hyperparameters

In [2]:
use_device='0'
device = torch.device('cuda:'+use_device if torch.cuda.is_available() else 'cpu')

In [3]:
fine_tune=True

In [4]:
## This can be either madgrad; onecyle
# training_config={'training_type':'feature_extractor','base_model_name':'sup_regular','layer_name':'features.4',
#                  'optimizer_name':'sgd','optimizer_lr':0.001,'optimizer_scheduler':True,'optimizer_wd':0.0,'total_num_epochs':100}

# training_config={'training_type':'feature_extractor','base_model_name':'sup_stylized','layer_name':'features.4',
#                  'optimizer_name':'sgd','optimizer_lr':0.001,'optimizer_scheduler':True,'optimizer_wd':0.0,'total_num_epochs':100}


# training_config={'training_type':'feature_extractor','base_model_name':'unsup_barlow','layer_name':'backbone.5',
#                  'optimizer_name':'sgd','optimizer_lr':0.001,'optimizer_scheduler':True,'optimizer_wd':0.0,'total_num_epochs':100}


# training_config={'training_type':'feature_extractor','base_model_name':'vit_regular','layer_name':'transformer.blocks.1.attn',
#                  'optimizer_name':'sgd','optimizer_lr':0.001,'optimizer_scheduler':True,'optimizer_wd':0.0,'total_num_epochs':100}


training_config={'training_type':'feature_extractor','base_model_name':'alexnet_bagnet9','layer_name':'features.4',
                 'optimizer_name':'sgd','optimizer_lr':0.001,'optimizer_scheduler':True,'optimizer_wd':0.0,'total_num_epochs':100}

In [5]:
base_model_name=training_config['base_model_name']
layer_name=training_config['layer_name']

# Splice Model Class - added in the python file

In [6]:
# class SpliceModel(nn.Module):
#     def __init__(self, base_model_name, layer_name,fine_tune=True, device='cpu'):
#         super().__init__()
        
#         self.base_model_name=base_model_name
#         self.layer_name=layer_name
#         self.fine_tune=fine_tune
#         self.device=device
#         self.temp_input_to_base=None
        
        
#         self.base_model=get_base_model(self.base_model_name).to(self.device)
#         self.arrange_base_model()
        
        
#         self.readout_model=get_readout_model(self.base_model,self.layer_name,temp_input_to_base=self.temp_input_to_base).to(self.device)
        
        
    
#     def arrange_base_model(self):
#         if(self.fine_tune):
#             ## Do nothing
#             pass
#         else:
#             ##freeze the weights of self.base model
#             freeze_all_parameters(self.base_model)
#             self.base_model.eval()
        
        
#         if('vit' not in self.base_model_name):
#             self.temp_input_to_base=torch.randn(2,3,512,512).to(self.device)

#         else:
#             self.temp_input_to_base=torch.randn(2,3,384,384).to(self.device)
            
            
    
    
    
#     def forward(self, x):
        
#         output=None
#         with FeatureExtractor(self.base_model, self.layer_name, detach=False, clone=True, retain=False) as extractor:

#             features = extractor(x)
#             temp_input_to_readout=features[self.layer_name].to(self.device)
#             if(len(temp_input_to_readout.shape)!=2):
#                 temp_input_to_readout=temp_input_to_readout.view((temp_input_to_readout.shape[0],-1))

#             output= self.readout_model(temp_input_to_readout)
#         return output
        

# Function to get only trainable parameters - added in the python file

In [7]:
# def get_untrainable_params(spliced_model):
    
#     assert isinstance(spliced_model,SpliceModel)
#     list_untrainable_parameters=[]
#     torch.mean(spliced_model.forward(spliced_model.temp_input_to_base)).backward()

#     for name,param in spliced_model.named_parameters():
#         if(param.requires_grad==False):
#             # All the frozen parameters
#             list_untrainable_parameters.append(name)
#         else:
#             ## This is for the fine-tune model parameters which are after the hooked layer so, not frozen by default but are unused and not updated
#             if(param.grad==None):
#                 list_untrainable_parameters.append(name)
    
#     return list_untrainable_parameters


# def get_modified_state_dict(spliced_model,list_untrainable_parameters):
#     temp_dict=spliced_model.state_dict()
#     for key in list_untrainable_parameters:
#         del temp_dict[key]
#     return temp_dict

# 1. Build the model

In [8]:
spliced_model=SpliceModel(base_model_name,layer_name,fine_tune=fine_tune,device=device)

... loading checkpoint: ../../dev/base_model_weights/alexnet_bagnet9_73111_final_weights_pytorch2-26c23b5440.pth


In [9]:
spliced_model

SpliceModel(
  (base_model): AlexNetBagnet9_73111(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1))
      (3): ReLU(inplace=True)
      (4): Conv2d(192, 384, kernel_size=(1, 1), stride=(1, 1))
      (5): ReLU(inplace=True)
      (6): Conv2d(384, 256, kernel_size=(1, 1), stride=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
      (9): ReLU(inplace=True)
    )
    (avgpool): Sequential(
      (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
      (pool2): AdaptiveAvgPool2d(output_size=(6, 6))
    )
    (classifier): Sequential(
      (0): Dropout(p=0.5, inplace=False)
      (1): Linear(in_features=9216, out_features=4096, bias=True)
      (2): ReLU(inplace=True)
      (3): Dropout(p=0.5, inplace=False)
      (4): Linear(in_features=4096, out_features=4096, bias=True)
      (5): ReLU(inplace=True)
   

# 2. Checking the parameters - Updated params, Saved params

In [ ]:
names_all_parameters=[]
names_all_parameters_requires_grad=[]
for name,param in spliced_model.named_parameters():
    names_all_parameters.append(name)
    if(param.requires_grad):
        names_all_parameters_requires_grad.append(name)


keys_all_state_dict=list(spliced_model.state_dict().keys())

In [ ]:
print(names_all_parameters)

print(names_all_parameters_requires_grad)

print(keys_all_state_dict)

# 3. Forward pass

In [ ]:
view_img=torch.randn(2,3,512,512).to(device)

# view_img=torch.randn(2,3,384,384).to(device)


spliced_model.forward(view_img).shape

# 4. The following cells need to be run sequenctially to check if gradients get saved or not, Skip this and go to 5

### 4A. Get input to the spliced model

In [ ]:
view_img=torch.randn(2,3,512,512).to(device)

# view_img=torch.randn(2,3,384,384).to(device)

### 4B. Check if gradients are saved yet, if just constructed, it shouldn't

In [ ]:
print(list(spliced_model.base_model.parameters())[0].grad.shape)

In [ ]:
print(list(spliced_model.readout_model.parameters())[0].grad.shape)

In [ ]:
print(list(spliced_model.parameters())[0].grad.shape)

### 4c. Get the output

In [ ]:
output=spliced_model.forward(view_img)

output.shape

### 4D. Check again if gradients are saved yet. Since errors are not propagated back, it shouldn't

In [ ]:
print(list(spliced_model.base_model.parameters())[0].grad.shape)

In [ ]:
print(list(spliced_model.readout_model.parameters())[0].grad.shape)

In [ ]:
print(list(spliced_model.parameters())[0].grad.shape)

### 4E. Get loss and propagate the errors back

In [ ]:
criterion=nn.MSELoss()
target=torch.randn_like(output)
loss=criterion(target,output)

loss.backward()



# torch.mean(output).backward()

### 4F. Check again if gradients are saved yet. Finally, errors have been propagated back, so should be saved

In [ ]:
print(list(spliced_model.base_model.parameters())[0].grad.shape)

In [ ]:
print(list(spliced_model.readout_model.parameters())[0].grad.shape)

In [ ]:
print(list(spliced_model.parameters())[0].grad.shape)

# 5. Saving the model

### Saving all the parameters

In [ ]:
torch.save({'model_state_dict':spliced_model.state_dict},'./model.pt')
!ls -lh ./model.pt

### Saving only the trainable parameters

In [ ]:
get_modified_state_dict(spliced_model,get_untrainable_params(spliced_model)).keys()

In [ ]:
torch.save({'base_model_name':base_model_name,
            'layer_name':layer_name,
            'fine_tune':True,
            'device':device,
            'model_state_dict':get_modified_state_dict(spliced_model,get_untrainable_params(spliced_model))},'./model.pt')
!ls -lh ./model.pt

# 6. Loading the model

In [ ]:
checkpoint=torch.load('./model.pt')

In [ ]:
loaded_spliced_model=SpliceModel(checkpoint['base_model_name'],checkpoint['layer_name'],fine_tune=checkpoint['fine_tune'],device=checkpoint['device'])
loaded_spliced_model.load_state_dict(checkpoint['model_state_dict'],strict=False)

In [ ]:
!rm -rf ./model.pt

# 7. Outputs from the loaded model

In [ ]:
view_img=torch.randn(2,3,512,512).to(checkpoint['device'])

loaded_spliced_model.forward(view_img).shape

In [ ]:
loaded_spliced_model

# 8. Swicthing model eval and train models [Function added to py file]

In [ ]:
print(spliced_model.training)

print(spliced_model.base_model.training)

print(spliced_model.readout_model.training)

print(fine_tune)

In [ ]:
# def change_train_eval_mode(spliced_model,fine_tune,train_eval_mode='eval'):
#     if(fine_tune):
#         if(train_eval_mode=='train'):
#             spliced_model.train()
#         elif(train_eval_mode=='eval'):
#             spliced_model.eval()
#     else:
#         if(train_eval_mode=='train'):
#             spliced_model.train()
#             spliced_model.base_model.eval()
#             # This is just for safety check, already true because whole sploiced model is in training mode
#             spliced_model.readout_model.train()
#         elif(train_eval_mode=='eval'):
#             spliced_model.eval()


In [ ]:
change_train_eval_mode(spliced_model,False,train_eval_mode='eval')

In [ ]:
print(spliced_model.training)

print(spliced_model.base_model.training)

print(spliced_model.readout_model.training)